# Visualization of Bicycle Infrared detectors across Hamburg

### Introduction

The city of Hamburg introduced in 2020 some infrared sensors to observe the circulation of bicycles accross different points of the city. This give the opportunity to the city to analyse the patterns on bicycle flows depending on availability of routes, weather and seasonality. In this project will focuse first on the visualization on the places where these sensors are placed and a first analysis of its location.

### Resources

As resource it will be used the data openly provided by the city of Hamburg.

A link to the source is provided here [Data Source](https://suche.transparenz.hamburg.de/dataset/verkehrsdaten-rad-infrarotdetektoren-hamburg8?forceWeb=true)

## Sensor location

### Data interpretation and extraction

It was decided to analyze first the location of the sensors. This analysis will give a better understanding of which parts of hamburg are being tracked by this devices and afterwards it could be related other factors to the lectures of the sensors.

In [157]:
#Needed libraries
import numpy as np
import pandas as pd
import math

##### First data exploration

In [158]:
#This is used to read the location data of the sensors
df_sensor_loc = pd.read_json("https://iot.hamburg.de/v1.1/Things?$filter=Datastreams/properties/serviceName%20eq%20%27HH_STA_HamburgerRadzaehlnetz%27%20&$count=true&$expand=Locations")

In [159]:
df_sensor_loc

,@iot.count,@iot.nextLink,value
0,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
1,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
2,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
3,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
4,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
...,...,...,...
95,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Zählfeld zur Bestimmung der v...
96,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Zählfeld zur Bestimmung der v...
97,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Zählfeld zur Bestimmung der v...
98,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Zählfeld zur Bestimmung der v...


The first display of the data doesn't tell many on the first view. Some of the insights found were:
1. There are 312 sensors around Hamburg, as noticed by the @iot.count column
2. This first extraction of the data showed only the first 100 sensors, to look at the next 100 the data showed in the column @iot.nestLink the address to the .json object for the next ones. 
3. The column value probably is the one containing the information of each sensor. This has to be further explored.

Now the information contained in the column value will be explored, to understand the data set.

In [160]:
#Display first value of the column value
sen_1 = df_sensor_loc['value'][0]
sen_1

{'description': 'Verkehrszählstelle zur Zählung der vom Infrarotdetektor erfassten Mobilitätswerkzeuge',
 '@iot.id': 5564,
 'name': 'Verkehrszählstelle 0295970',
 'properties': {'assetID': '0295970',
  'keywords': ['Infrarotdetektor',
   'Verkehrsmenge',
   'automatisierte Verkehrsmengenerfassung',
   'Hamburger Radzählnetz',
   'Hamburg',
   'aVME',
   'HaRaZäN',
   'Zählstelle'],
  'language': 'de',
  'richtung': 'Querschnitt',
  'ownerThing': 'Freie und Hansestadt Hamburg',
  'infoLastUpdate': '2021-09-16T07:59:31.678Z'},
 '@iot.selfLink': 'https://iot.hamburg.de/v1.1/Things(5564)',
 'Locations': [{'description': 'Mittelpunkt des Streckenverlaufes bezüglich Zählstelle 0295 auf dem Richtungsarm Südwest für den Zählquerschnitt',
   'encodingType': 'application/vnd.geo+json',
   '@iot.id': 26517652,
   'location': {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [9.999098, 53.579928]}},
   'name': 'Verkehrszählstelle 0295970',
   '@iot.selfLink': 'https://iot.hambur

The column value contains a directiory with different information of a particular snesor. For this first part of the project it was important a unique id number (@iot.id), and the location. It was also notiuced that this directory contains nested directories like in the case of the location.

In [161]:
#Extraction of the keys of the directory to further explore the location
sen_1.keys()

dict_keys(['description', '@iot.id', 'name', 'properties', '@iot.selfLink', 'Locations', 'HistoricalLocations@iot.navigationLink', 'MultiDatastreams@iot.navigationLink', 'TaskingCapabilities@iot.navigationLink', 'Locations@iot.navigationLink', 'Datastreams@iot.navigationLink'])

In [162]:
#Display information contained by key 'Locations'
sen_1['Locations']

[{'description': 'Mittelpunkt des Streckenverlaufes bezüglich Zählstelle 0295 auf dem Richtungsarm Südwest für den Zählquerschnitt',
  'encodingType': 'application/vnd.geo+json',
  '@iot.id': 26517652,
  'location': {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [9.999098, 53.579928]}},
  'name': 'Verkehrszählstelle 0295970',
  '@iot.selfLink': 'https://iot.hamburg.de/v1.1/Locations(26517652)'}]

In the previous cell it can be seen the information contained by the key 'Locations', which includes the coordinates of the sensor's location. A possibility to manually extract the coordinates could be by typing the following: sen_1['Locations'][0]['location']['geometry']['coordinates']

As already mentioned before, the information of the sensors is stored separately and it can show a maximum of 100 per data set. Therefore in the next part all this information will be gathered and stored in a single data frame.

In [163]:
#Function definition to store all data in the same data frame
def data_build(data, n):
    for i in range(1,n):
        j = data.shape[0]-1
        url = data['@iot.nextLink'][j]
        new_df = pd.read_json(url)
        data = data.append(new_df, ignore_index = True)
    return(data)

In [164]:
max_sen = df_sensor_loc['@iot.count'][0] #extraction total amount of sensors
n = math.ceil(max_sen/100) #definition of loop numbers
df_sensor_loc = data_build(df_sensor_loc, n) #call function to store all the data in same data frame

In [165]:
df_sensor_loc #Display of the merging result

,@iot.count,@iot.nextLink,value
0,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
1,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
2,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
3,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
4,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,{'description': 'Verkehrszählstelle zur Zählun...
...,...,...,...
307,312,NaN,{'description': 'Zählfeld zur Bestimmung der v...
308,312,NaN,{'description': 'Zählfeld zur Bestimmung der v...
309,312,NaN,{'description': 'Zählfeld zur Bestimmung der v...
310,312,NaN,{'description': 'Zählfeld zur Bestimmung der v...


Now the data in value column will be extracted and a more user friendly data frame will be created for a posterior visualization. At this point the columns @iot.count and @iot.nextLink won't help at all any more, therefore we will replace our actual data frame for values in the column value, which will be normalized.

In [166]:
#Normalization of the data set
df_sensor_loc = pd.json_normalize(df_sensor_loc['value'])
df_sensor_loc.head()

,description,@iot.id,name,@iot.selfLink,Locations,HistoricalLocations@iot.navigationLink,MultiDatastreams@iot.navigationLink,TaskingCapabilities@iot.navigationLink,Locations@iot.navigationLink,Datastreams@iot.navigationLink,properties.assetID,properties.keywords,properties.language,properties.richtung,properties.ownerThing,properties.infoLastUpdate,properties.topic
0,Verkehrszählstelle zur Zählung der vom Infraro...,5564,Verkehrszählstelle 0295970,https://iot.hamburg.de/v1.1/Things(5564),[{'description': 'Mittelpunkt des Streckenverl...,https://iot.hamburg.de/v1.1/Things(5564)/Histo...,https://iot.hamburg.de/v1.1/Things(5564)/Multi...,https://iot.hamburg.de/v1.1/Things(5564)/Taski...,https://iot.hamburg.de/v1.1/Things(5564)/Locat...,https://iot.hamburg.de/v1.1/Things(5564)/Datas...,0295970,"[Infrarotdetektor, Verkehrsmenge, automatisier...",de,Querschnitt,Freie und Hansestadt Hamburg,2021-09-16T07:59:31.678Z,NaN
1,Verkehrszählstelle zur Zählung der vom Infraro...,5565,Verkehrszählstelle 0295971,https://iot.hamburg.de/v1.1/Things(5565),[{'description': 'Mittelpunkt des Streckenverl...,https://iot.hamburg.de/v1.1/Things(5565)/Histo...,https://iot.hamburg.de/v1.1/Things(5565)/Multi...,https://iot.hamburg.de/v1.1/Things(5565)/Taski...,https://iot.hamburg.de/v1.1/Things(5565)/Locat...,https://iot.hamburg.de/v1.1/Things(5565)/Datas...,0295971,"[Infrarotdetektor, Verkehrsmenge, automatisier...",de,Nordost nach Südwest,Freie und Hansestadt Hamburg,2021-09-16T07:59:30.886Z,NaN
2,Verkehrszählstelle zur Zählung der vom Infraro...,5566,Verkehrszählstelle 0295972,https://iot.hamburg.de/v1.1/Things(5566),[{'description': 'Mittelpunkt des Streckenverl...,https://iot.hamburg.de/v1.1/Things(5566)/Histo...,https://iot.hamburg.de/v1.1/Things(5566)/Multi...,https://iot.hamburg.de/v1.1/Things(5566)/Taski...,https://iot.hamburg.de/v1.1/Things(5566)/Locat...,https://iot.hamburg.de/v1.1/Things(5566)/Datas...,0295972,"[Infrarotdetektor, Verkehrsmenge, automatisier...",de,Südwest nach Nordost,Freie und Hansestadt Hamburg,2021-09-16T07:59:30.014Z,NaN
3,Verkehrszählstelle zur Zählung der vom Infraro...,5567,Verkehrszählstelle 0004940,https://iot.hamburg.de/v1.1/Things(5567),[{'description': 'Mittelpunkt des Streckenverl...,https://iot.hamburg.de/v1.1/Things(5567)/Histo...,https://iot.hamburg.de/v1.1/Things(5567)/Multi...,https://iot.hamburg.de/v1.1/Things(5567)/Taski...,https://iot.hamburg.de/v1.1/Things(5567)/Locat...,https://iot.hamburg.de/v1.1/Things(5567)/Datas...,0004940,"[Infrarotdetektor, Verkehrsmenge, automatisier...",de,Querschnitt,Freie und Hansestadt Hamburg,2021-09-16T09:11:25.563Z,NaN
4,Verkehrszählstelle zur Zählung der vom Infraro...,5568,Verkehrszählstelle 0004941,https://iot.hamburg.de/v1.1/Things(5568),[{'description': 'Mittelpunkt des Streckenverl...,https://iot.hamburg.de/v1.1/Things(5568)/Histo...,https://iot.hamburg.de/v1.1/Things(5568)/Multi...,https://iot.hamburg.de/v1.1/Things(5568)/Taski...,https://iot.hamburg.de/v1.1/Things(5568)/Locat...,https://iot.hamburg.de/v1.1/Things(5568)/Datas...,0004941,"[Infrarotdetektor, Verkehrsmenge, automatisier...",de,Ost nach West,Freie und Hansestadt Hamburg,2021-09-16T09:11:22.559Z,NaN


A second data frame with the dictionary contained in locations will be created.

In [167]:
df_loc = df_sensor_loc['Locations'].apply(lambda x: x[0]) #Extraction of dictionaries stored in column 'Locations'
df_loc = pd.json_normalize(df_loc) # Normalization of the dictionary into a data frame

In [168]:
#spliting of the 'location.geometry.coordinates' into 'Longitude' and 'Latitude' columns
df_loc[['Logitude', 'Latitude']] = pd.DataFrame(df_loc['location.geometry.coordinates']
                                                .tolist(), index=df_loc.index) 
df_loc.head()

,description,encodingType,@iot.id,name,@iot.selfLink,location.type,location.geometry.type,location.geometry.coordinates,Logitude,Latitude
0,Mittelpunkt des Streckenverlaufes bezüglich Zä...,application/vnd.geo+json,26517652,Verkehrszählstelle 0295970,https://iot.hamburg.de/v1.1/Locations(26517652),Feature,Point,"[9.999098, 53.579928]",9.999098,53.579928
1,Mittelpunkt des Streckenverlaufes bezüglich Zä...,application/vnd.geo+json,26517653,Verkehrszählstelle 0295971,https://iot.hamburg.de/v1.1/Locations(26517653),Feature,Point,"[9.998994, 53.579992]",9.998994,53.579992
2,Mittelpunkt des Streckenverlaufes bezüglich Zä...,application/vnd.geo+json,26517654,Verkehrszählstelle 0295972,https://iot.hamburg.de/v1.1/Locations(26517654),Feature,Point,"[9.999202, 53.579864]",9.999202,53.579864
3,Mittelpunkt des Streckenverlaufes bezüglich Zä...,application/vnd.geo+json,26517658,Verkehrszählstelle 0004940,https://iot.hamburg.de/v1.1/Locations(26517658),Feature,Point,"[9.953765, 53.575705]",9.953765,53.575705
4,Mittelpunkt des Streckenverlaufes bezüglich Zä...,application/vnd.geo+json,26517659,Verkehrszählstelle 0004941,https://iot.hamburg.de/v1.1/Locations(26517659),Feature,Point,"[9.95378679, 53.5757405]",9.953787,53.575741


##### Second option to try is to use .json_normalize

**Create the location data frame**

In [12]:
df_sen_loc = pd.DataFrame()

In [13]:
df_sen_loc['Name'] = test_loc['name'] 

In [14]:
test_org = df_sen_loc.Name.str.split(expand=True)

In [15]:
list_type = []
for i,row in test_org.iterrows():
    if row[6] == None:
        row[1], row[6] = row[6], row[1]
        list_type.append(row[0]) 
    else:
        row[3], row[4], row[5], row[6] = row[4], row[5], row[6], row[3]
        list_type.append((row[0] + ' ' + row[1] + ' ' 
                    + row[2] + ' ' + row[3] + ' ' + row[4] + ' ' + row[5]))
        #print(row[6]==None)
        #print(row)
        #print(i)

In [16]:
test_org['Type'] = list_type

In [17]:
del test_org[0]
del test_org[1]
del test_org[2]
del test_org[3]
del test_org[4]
del test_org[5]

In [18]:
test_org.columns = ['ID', 'Type']

In [19]:
test_org['Loc'] = test_loc['location.geometry.coordinates']
test_org[['Logitude', 'Latitude']] = pd.DataFrame(test_org.Loc.tolist(), index=df_sen_loc.index)
test_org

,ID,Type,Loc,Logitude,Latitude
0,0295970,Verkehrszählstelle,"[9.999098, 53.579928]",9.999098,53.579928
1,0295971,Verkehrszählstelle,"[9.998994, 53.579992]",9.998994,53.579992
2,0295972,Verkehrszählstelle,"[9.999202, 53.579864]",9.999202,53.579864
3,0004940,Verkehrszählstelle,"[9.953765, 53.575705]",9.953765,53.575705
4,0004941,Verkehrszählstelle,"[9.95378679, 53.5757405]",9.953787,53.575741
...,...,...,...,...,...
95,D_22.2_1_G,Ankerpunkt des Zählfeldes des zugeordneten Inf...,"[9.868206, 53.571221]",9.868206,53.571221
96,D_22.2_2_I,Ankerpunkt des Zählfeldes des zugeordneten Inf...,"[9.868206, 53.571221]",9.868206,53.571221
97,B_16.1_1_I,Ankerpunkt des Zählfeldes des zugeordneten Inf...,"[10.018952, 53.490956]",10.018952,53.490956
98,B_16.1_2_G,Ankerpunkt des Zählfeldes des zugeordneten Inf...,"[10.018952, 53.490956]",10.018952,53.490956


##### Request API test

In [20]:
import requests

In [21]:
#url = 'https://iot.hamburg.de/v1.1/Datastreams?$skip=100'
url = 'https://iot.hamburg.de/v1.1/Things?$filter=Datastreams/properties/serviceName%20eq%20%27HH_STA_HamburgerRadzaehlnetz%27%20&$count=true&$expand=Locations'
response = requests.get(url).json()
data = pd.json_normalize(response)

In [22]:
data

,@iot.count,@iot.nextLink,value
0,312,https://iot.hamburg.de/v1.1/Things?$skip=100&$...,[{'description': 'Verkehrszählstelle zur Zählu...


In [23]:
data['value'][0][99]

{'description': 'Zählfeld zur Bestimmung der vom Infrarotdetektor erfassten Fahrräder',
 '@iot.id': 5715,
 'name': 'Zählfeld B_50.6_1_I',
 'properties': {'assetID': 'B_50.6_1_I',
  'keywords': ['Infrarotdetektor',
   'Verkehrsmenge',
   'automatisierte Verkehrsmengenerfassung',
   'Hamburger Radzählnetz',
   'Hamburg',
   'aVME',
   'HaRaZäN',
   'Zählfeld'],
  'language': 'de',
  'richtung': 'Hauptrichtung',
  'ownerThing': 'Hamburg Verkehrsanlagen',
  'infoLastUpdate': '2021-09-16T09:08:22.899Z'},
 '@iot.selfLink': 'https://iot.hamburg.de/v1.1/Things(5715)',
 'Locations': [{'description': 'Zählfeld auf Geh-/Radweg in Richtung 1 in Hauptrichtung zur Hauptverkehrsführung',
   'encodingType': 'application/vnd.geo+json',
   '@iot.id': 26517494,
   'location': {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [9.98469105, 53.4569609]}},
   'name': 'Ankerpunkt des Zählfeldes B_50.6_1_I des zugeordneten Infrarotdetektors',
   '@iot.selfLink': 'https://iot.hamburg.de/v1.1/

In [24]:
data['@iot.nextLink'][0]

'https://iot.hamburg.de/v1.1/Things?$skip=100&$filter=%28Datastreams%2Fproperties%2FserviceName+eq+%27HH_STA_HamburgerRadzaehlnetz%27%29&$expand=Locations&$count=true'

In [25]:
url = data['@iot.nextLink'][0]
response = requests.get(url).json()
data = pd.json_normalize(response)

In [26]:
data

,@iot.count,@iot.nextLink,value
0,312,https://iot.hamburg.de/v1.1/Things?$skip=200&$...,[{'description': 'Zählfeld zur Bestimmung der ...


In [27]:
data['value'][0][11]

{'description': 'Zählfeld zur Bestimmung der vom Infrarotdetektor erfassten Fahrräder',
 '@iot.id': 5727,
 'name': 'Zählfeld A_25.1_1_I',
 'properties': {'assetID': 'A_25.1_1_I',
  'keywords': ['Infrarotdetektor',
   'Verkehrsmenge',
   'automatisierte Verkehrsmengenerfassung',
   'Hamburger Radzählnetz',
   'Hamburg',
   'aVME',
   'HaRaZäN',
   'Zählfeld'],
  'language': 'de',
  'richtung': 'Hauptrichtung',
  'ownerThing': 'Hamburg Verkehrsanlagen',
  'infoLastUpdate': '2021-09-16T09:08:06.003Z'},
 '@iot.selfLink': 'https://iot.hamburg.de/v1.1/Things(5727)',
 'Locations': [{'description': 'Zählfeld auf Radweg in Richtung 1 in Hauptrichtung zur Hauptverkehrsführung',
   'encodingType': 'application/vnd.geo+json',
   '@iot.id': 26517506,
   'location': {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [9.998648, 53.557595]}},
   'name': 'Ankerpunkt des Zählfeldes A_25.1_1_I des zugeordneten Infrarotdetektors',
   '@iot.selfLink': 'https://iot.hamburg.de/v1.1/Location